# Identificar y remover ruido con RegEx

**Cargar datos de Reddit Self-post. Fuente: [Kaggle](https://www.kaggle.com/datasets/mswarbrickjones/reddit-selfposts)**

In [4]:
import pandas as pd 

In [5]:
posts_file = "data/rspct_autos.tsv.gz"
posts_df = pd.read_csv(posts_file, sep='\t')

In [6]:
posts_df

,id,subreddit,title,selftext
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ..."
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu..."
...,...,...,...,...
19995,7i2k6y,4Runner,Bilstein Shocks,I read a lot Forums and people recommend getti...
19996,83p2kv,Harley,Question on potential purchase of crashed bike.,I am thinking about buying a 2010 Harley Spor...
19997,7x722h,volt,Got our first warning light on our dash,My husband and I were headed somewhere and I w...
19998,7v2xmg,Lexus,Any IS models to avoid?,I am looking at getting a used Lexus IS (2014 ...


In [7]:
subred_file = "data/subreddit_info.csv.gz"
subred_df = pd.read_csv(subred_file).set_index(['subreddit'])

df = posts_df.join(subred_df, on='subreddit')

In [8]:
df.head()

,id,subreddit,title,selftext,category_1,category_2,category_3,in_data,reason_for_exclusion
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson,NaN,True,NaN
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ...",autos,ford,NaN,True,NaN
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...,autos,VW,NaN,True,NaN
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...,autos,lexus,NaN,True,NaN
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu...",autos,chevrolet,NaN,True,NaN


In [9]:
print(df.columns)

Index(['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2',
       'category_3', 'in_data', 'reason_for_exclusion'],
      dtype='object')


In [10]:
column_mapping = {
    'id' : 'id',
    'subreddit' : 'subreddit',
    'title' : 'title',
    'selftext' : 'text',
    'category_1' : 'category',
    'category_2' : 'subcategory',
    'category_3' : None, #no data 
    'in_data' : None, #not needed
    'reason_for_exlusion': None #not needed
}

In [11]:
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]

In [12]:
df = df[columns].rename(columns=column_mapping)

In [13]:
df = df[df['category'] == 'autos']
df.sample(1).T

,17273
id,4xda3l
subreddit,subaru
title,Is this necessary or a dealership cash grab?
text,http://i.imgur.com/euoiQo3.jpg<lb><lb>I had so...
category,autos
subcategory,subaru


**Guardar y cargar datos a un DataFrame**

In [14]:
import sqlite3

db_name = "reddit_selfpost.db"
con = sqlite3.connect(db_name)
df.to_sql("posts", con, index=False, if_exists="replace")
con.close()

In [15]:
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from posts", con)
con.close()

**Identificación de ruido con regex**


In [16]:
import re

RE_SUSPICIUS = re.compile(r'[&#<>{}\[\]\\]')

def impurity(text, min_len=10):
    if text == None or len(text)< min_len:
        return 0
    else:
        return len(RE_SUSPICIUS.findall(text))/len(text)

In [17]:
df['impurity'] = df['text'].apply(impurity, min_len=10)

In [18]:
df[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(10)

,text,impurity
19682,Looking at buying a 335i with 39k miles and 11...,0.214716
12357,I'm looking to lease an a4 premium plus automa...,0.165099
2730,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L...,0.139130
12754,Bulbs Needed:<lb><lb><lb>**194 LED BULB x8**<l...,0.132411
10726,I currently have a deposit on a 2013 335is (CP...,0.129317
11122,"Vehicle Price<tab><tab>$40,650.00<tab> <lb> <t...",0.119777
16895,TITLE IS WRONG IT'S A 2014<lb><lb>Interested i...,0.116208
2989,Looking into sport trucks and these are both i...,0.111570
1036,Subaru 1.5r aka Felicity<lb><lb>Fairly stock a...,0.111111
12303,I'm having a bit of difficulty deciding betwee...,0.108352


In [19]:
import pandas as pd
from tqdm import tqdm
from collections import Counter

tqdm.pandas()  # Habilita barra de progreso en pandas

def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].progress_apply(update)  # <--- corrección aquí

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    return freq_df.sort_values('freq', ascending=False)

In [20]:
count_words(df, column='text', preprocess=lambda t : re.findall(r'<[\w/]*>', t))

100%|██████████| 20000/20000 [00:00<00:00, 464292.68it/s]


,freq
token,
<lb>,100729
<tab>,642


**Remover ruido con regex**

In [21]:
import html

def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [24]:
df['clean_text'] = df['text'].map(clean)
df['impurity'] = df['clean_text'].apply(impurity, min_len=20)
df[['clean_text', 'impurity']].sort_values(by='impurity', ascending=False).head(10)

,clean_text,impurity
14058,"Mustang 2018, 2019, or 2020? Must Haves!! 1. H...",0.030864
18934,"At the dealership, they offered an option for ...",0.026455
16505,"I am looking at four Caymans, all are in a sim...",0.024631
4376,"Hello, I came across this great looking e500 a...",0.022762
18729,The Mazda 3 hatchback I'm looking at is a 2007...,0.022099
15505,Not sure if this type of post is allowed but.....,0.021938
3255,I am in need of two front tires and instead of...,0.021792
11776,"Hi, I'm looking to buy an A45 AMG \(here in th...",0.021251
14935,"I heard these cars are money pits, but this on...",0.020927
19249,My car broke down this week driving up to Davi...,0.019274
